# Getting ORES scores through LiftWing ML Service API

## Disclaimer
Wikimedia Foundation (WMF) is reworking access to their APIs. It is likely in the coming years that all API access will require some kind of authentication, either through a simple key/token or through some version of OAuth. For now this is still a work in progress. You can follow the progress from their [API portal](https://api.wikimedia.org/wiki/Main_Page). Another on-going change is better control over API services in situations where those services require additional computational resources, beyond simply serving the text of a web page (i.e., the text of an article). Services like ORES that require running an ML model over the text of an article page is an example of a compute intensive API service.

Wikimedia is implementing a new Machine Learning (ML) service infrastructure that they call [LiftWing](https://wikitech.wikimedia.org/wiki/Machine_Learning/LiftWing). Given that ORES already has several ML models that have been well used, ORES is the first set of APIs that are being moved to LiftWing.

The code uses article quality estimates for article revisions using the LiftWing version of [ORES](https://www.mediawiki.org/wiki/ORES). The [ORES API documentation](https://ores.wikimedia.org) can be accessed from the main ORES page.


## License
This notebook uses parts of code and comments which were developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - August 15, 2023



In [9]:
#
# These are standard python modules
import json, time, urllib.parse
from dotenv import load_dotenv
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests
import Constants as cs
import pandas as pd

Please note that in order to run the code form here you will need to create a file named Constants.py and add the following variables to it:

```
WIKIPEDIA_USERNAME = "your wikipedia username here"

WIKIPEDIA_ACCESS_TOKEN = "your wikipedia access token here"
```

Use below infomation to get your own access token. 

## DANGER: THE CODE WILL NOT WORK WITHOUT THE Constants.py file.

### How to get your own access token

You will need a Wikimedia user account to get access to Lift Wing (the ML API service). You can either [create an account or login](https://api.wikimedia.org/w/index.php?title=Special:UserLogin). If you have a Wikipedia user account - you might already have an Wikimedia account. If you are not sure try your Wikipedia username and password to check it. If you do not have a Wikimedia account you will need to create an account that you can use to get an access token.

There is [a 'guide' that describes how to get authentication tokens](https://api.wikimedia.org/wiki/Authentication) - but not everything works the way it is described in that documentation. You should review that documentation and then read the rest of this comment.

The documentation talks about using a "dashboard" for managing authentication tokens. That's a rather generous description for what looks like a simple list of token things. You might have a hard time finding this "dashboard". First, on the left hand side of the page, you'll see a column of links. The bottom section is a set of links titled "Tools". In that section is a link that says [Special pages](https://api.wikimedia.org/wiki/Special:SpecialPages) which will take you to a list of ... well, special pages. At the very bottom of the "Special pages" page is a section titled "Other special pages" (scroll all the way to the bottom). The first link in that section is called [API keys](https://api.wikimedia.org/wiki/Special:AppManagement). When you get to the "API keys" page you can create a new key.

The authentication guide suggests that you should create a server-side app key. This does not seem to work correctly - as yet. It failed on multiple attempts when I attempted to create a server-side app key. BUT, there is an option to create a [Personal API token](https://api.wikimedia.org/wiki/Authentication) that should work for this course and the type of ORES page scoring that you will need to perform.


In [6]:
#########
#
#    CONSTANTS
#

#    The current LiftWing ORES API endpoint and prediction model
#
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = ((60.0*60.0)/5000.0)-API_LATENCY_ASSUMED  # The key authorizes 5000 requests per hour

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "dabhinav@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2024",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}
#
#    This is a template for the parameters that we need to supply in the headers of an API request
#
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}

#
#    A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
#
ARTICLE_REVISIONS = { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

#
#    These are used later - defined here so they, at least, have empty values
#
USERNAME = cs.WIKIPEDIA_USERNAME
ACCESS_TOKEN = cs.WIKIPEDIA_ACCESS_TOKEN

The Wikimedia Foundation appears to be issuing access tokens that are adhering to the [JWT (JSON Web Token) standard](https://jwt.io/introduction/). There was also some documentation by IBM about the [use of JWT tokens](https://www.ibm.com/docs/en/cics-ts/6.1?topic=cics-json-web-token-jwt) that I found useful. Keep in mind, documentation from IBM is specific to their implementation of the JWT standard. Access tokens are composed of different parts that specify the domain being accessed and rate limits. The little snippet of code below is not required to make ORES requests. It just allows us to see what is in the Wikimedia provided access token that you were issued.

## Define a function to make the ORES API request

The API request will be made using a function to encapsulate call and make access reusable in other notebooks. The procedure is parameterized, relying on the constants above for some important default parameters. The primary assumption is that this function will be used to request data for a set of article revisions. The main parameter is 'article_revid'. One should be able to simply pass in a new article revision id on each call and get back a python dictionary as the result. A valid result will be a dictionary that contains the probabilities that the specific revision is one of six different article quality levels. Generally, quality level with the highest probability score is considered the quality level for the article. This can be tricky when you have two (or more) highly probable quality levels.

In [7]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT,
                                   model_name = API_ORES_EN_QUALITY_MODEL,
                                   request_data = ORES_REQUEST_DATA_TEMPLATE,
                                   header_format = REQUEST_HEADER_TEMPLATE,
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):

    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token

    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")

    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)

    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


We need the rev_ids we got from the previous notebook to get the quality of the articles from ORES. FOr this we read the csv file and get the rev_ids from it.


### Make sure to copy the csv file from the data_files folder and place it in the path of this notebook.

In [10]:
wiki_info = pd.read_csv('wiki_information.csv')
wiki_info


,name,revid
0,Abdul Baqi Turkistani,1.231655e+09
1,Abdul Ghani Ghani,1.227026e+09
2,Abdul Rahim Ayoubi,1.226326e+09
3,Ahmad Wali Massoud,1.221721e+09
4,Aimal Faizi,1.185106e+09
...,...,...
7150,Denis Walker,1.247903e+09
7151,Herbert Ushewokunze,9.591118e+08
7152,Josiah Tongogara,1.203429e+09
7153,Langton Towungana,1.246280e+09


Here, wiki_info['revid'] will contain a list of all the rev_ids.

In [ ]:
list_of_outputs = []
counter = 0

for article in list(wiki_info['revid']):
  print(f"Getting LiftWing ORES scores for '{article}' with revid: {article}")
#
#    Make the call, just pass in the article revision ID, email address, and access token
  score = request_ores_score_per_article(article_revid=int(article),
                                       email_address="dvabhinav31@gmail.com",
                                       access_token=ACCESS_TOKEN)
  print(counter)
  counter+=1
  list_of_outputs.append(json.dumps(score,indent=4))

Cell outputs above have been removed since they utilize too much space. Rerun them to get the response.

Next, we convert the string for each json into json and write it into a json file. 

Following is the structure of the json file: 
```
{ 
  "outputs": [
    {
      "enwiki": {
        "models": { "articlequality": { "version": "0.9.2" } },
        "scores": {
          "1231655023": {
            "articlequality": {
              "score": {
                "prediction": "Stub",
                "probability": {
                  "B": 0.007563164541978764,
                  "C": 0.010571998067933679,
                  "FA": 0.0014567448768872152,
                  "GA": 0.00350824677167893,
                  "Start": 0.04243220742433865,
                  "Stub": 0.9344676383171826
                }
              }
            }
          }
        }
      }
    },
    # many more such articles
  ]
}
```

In [11]:
#convert string to json
list_of_outputs = [json.loads(x) for x in list_of_outputs]

In [12]:
#write to json
with open("ores_output.json","w") as file:
    json.dump(list_of_outputs,file)